# NYC vs LA – defining similar neighborhoods

## 1.Introduction

NYC and LA - No two cities in the states are compared as much as these two.  One might prefer living in LA for many reasons but others will choose to live in NYC for almost same amount of reasons. Millions of articles have been published comparing many different aspects of these two Cities. But for this project, we are going to focus on similarities rather than differences. Our goal is to explain a New Yorker, who has never been in LA, about neighborhoods in LA by highlighting their similarities with neighborhoods in in NYC, or vice versa. 

## 2. Data

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
import matplotlib.pyplot as plt
%matplotlib inline 
print('Libraries imported.')

Libraries imported.


#### a.NYC neighborhood Zipcode, Latitude, Longitude data 

In [2]:
with open('nyu-2451-34572-geojson.json') as json_data:
    ny_data = json.load(json_data)

In [3]:
ny_data = ny_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny_df= pd.DataFrame(columns=column_names)


for data in ny_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_df = ny_df.append({'Borough': borough,
                        'Neighborhood': neighborhood_name,
                        'Latitude': neighborhood_lat,
                         'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
ny_df.drop('Borough', axis =1, inplace = True)
ny_df.insert(0,column = "City", value = "NY")
ny_df.head()

,City,Neighborhood,Latitude,Longitude
0,NY,Wakefield,40.894705,-73.847201
1,NY,Co-op City,40.874294,-73.829939
2,NY,Eastchester,40.887556,-73.827806
3,NY,Fieldston,40.895437,-73.905643
4,NY,Riverdale,40.890834,-73.912585


#### b.LA neighborhood Zipcode data

In [5]:
List_url = " http://www.laalmanac.com/communications/cm02a90001-90899.php"
source = requests.get(List_url).text
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [6]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

In [7]:
list_table = tableDataText(table)

In [8]:
column_names = ['Zipcode','Neighborhood']
la_df = pd.DataFrame(list_table[1:], columns=column_names)
la_df.head(4)

,Zipcode,Neighborhood
0,90001,"Los Angeles(South Los Angeles), Florence-Graham"
1,90002,"Los Angeles(Southeast Los Angeles, Watts)"
2,90003,"Los Angeles(South Los Angeles, Southeast Los A..."
3,90004,"Los Angeles(Hancock Park, Rampart Village, Vir..."


In [9]:
la_df.shape

(258, 2)

In [10]:
la_df['Zipcode'] = la_df.Zipcode.astype('int64')
la_df.dtypes

Zipcode          int64
Neighborhood    object
dtype: object

#### c.Latitude / Longitude information for LA

In [11]:
la_geo = pd.read_excel('C:/Users/hanah/Desktop/projects/us-zip-code-latitude-and-longitude.xlsx')

In [12]:
la_geo.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,92232,Calexico,CA,33.026203,-115.284581,-8,1,"33.026203, -115.284581"
1,93227,Goshen,CA,36.357151,-119.425371,-8,1,"36.357151, -119.425371"
2,93234,Huron,CA,36.209815,-120.084700,-8,1,"36.209815, -120.0847"
3,93529,June Lake,CA,37.765218,-119.077690,-8,1,"37.765218, -119.07769"
4,93761,Fresno,CA,36.746375,-119.639658,-8,1,"36.746375, -119.639658"


In [30]:
la_df2 = pd.merge(la_df, la_geo, how = 'left', left_on = "Zipcode", right_on = "Zip")
la_df2.head()

,Zipcode,Neighborhood,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,90001,"Los Angeles(South Los Angeles), Florence-Graham",90001.0,Los Angeles,CA,33.972914,-118.24878,-8.0,1.0,"33.972914, -118.24878"
1,90002,"Los Angeles(Southeast Los Angeles, Watts)",90002.0,Los Angeles,CA,33.948315,-118.24845,-8.0,1.0,"33.948315, -118.24845"
2,90003,"Los Angeles(South Los Angeles, Southeast Los A...",90003.0,Los Angeles,CA,33.962714,-118.27600,-8.0,1.0,"33.962714, -118.276"
3,90004,"Los Angeles(Hancock Park, Rampart Village, Vir...",90004.0,Los Angeles,CA,34.077110,-118.30755,-8.0,1.0,"34.07711, -118.30755"
4,90005,"Los Angeles(Hancock Park, Koreatown, Wilshire ...",90005.0,Los Angeles,CA,34.058911,-118.30848,-8.0,1.0,"34.058911, -118.30848"


In [14]:
la_df2.shape

(258, 10)

In [31]:
# filter out NAs -- 4 rows dropped 
la_df2.dropna(inplace =True)
# filter out poboxes -- 76rows dropped 
la_df2 = la_df2[la_df2['Neighborhood'].str.contains("PO Boxes")==False]
la_df2 = la_df2[['Neighborhood','Latitude','Longitude']]
la_df2.insert(0,column = "City", value = "LA")
la_df2.head()

,City,Neighborhood,Latitude,Longitude
0,LA,"Los Angeles(South Los Angeles), Florence-Graham",33.972914,-118.24878
1,LA,"Los Angeles(Southeast Los Angeles, Watts)",33.948315,-118.24845
2,LA,"Los Angeles(South Los Angeles, Southeast Los A...",33.962714,-118.27600
3,LA,"Los Angeles(Hancock Park, Rampart Village, Vir...",34.077110,-118.30755
4,LA,"Los Angeles(Hancock Park, Koreatown, Wilshire ...",34.058911,-118.30848


#### d. Foursquare API geolocation data 

In [18]:
# calculate distance between two closest data points for LA 
from geopy.distance import great_circle

#function to get the closest postcode pair and the distance using latitude and longitude
def closest_postcode(postcode, postcodes):
    postcode = np.asarray(postcode).reshape(1,-1)
    postcodes = np.asarray(postcodes)
    distances = [great_circle(postcode, point).meters for point in postcodes]
    closest_postcode_index = np.argmin(distances)
    return [round(distances[closest_postcode_index]), closest_postcode_index]

In [ ]:
coordinates = la_df2[['Latitude', 'Longitude']]
distance, closest_point_index = closest_postcode(coordinates[:1], coordinates[~coordinates.index.isin([0])])
close_points = coordinates[:1].append(coordinates[closest_point_index+1:closest_point_index+2])

In [37]:
from math import isnan

for lat, lng, idx in zip(la_df2['Latitude'], la_df2['Longitude'], la_df2.index):
    distance, closest_point_index = closest_postcode([lat,lng], coordinates[~coordinates.index.isin([idx])])
    la_df2.at[idx,'Distance'] = np.int64(distance//2) #use the half of the disstance to avoid overlapping
    
la_df2.head(5)

,City,Neighborhood,Latitude,Longitude,Distance
0,LA,"Los Angeles(South Los Angeles), Florence-Graham",33.972914,-118.24878,1367.0
1,LA,"Los Angeles(Southeast Los Angeles, Watts)",33.948315,-118.24845,1159.0
2,LA,"Los Angeles(South Los Angeles, Southeast Los A...",33.962714,-118.27600,872.0
3,LA,"Los Angeles(Hancock Park, Rampart Village, Vir...",34.077110,-118.30755,594.0
4,LA,"Los Angeles(Hancock Park, Koreatown, Wilshire ...",34.058911,-118.30848,360.0


In [38]:
coordinates = ny_df[['Latitude', 'Longitude']]

distance, closest_point_index = closest_postcode(coordinates[:1], coordinates[~coordinates.index.isin([0])])
close_points = coordinates[:1].append(coordinates[closest_point_index+1:closest_point_index+2])

In [39]:
from math import isnan

for lat, lng, idx in zip(ny_df['Latitude'], ny_df['Longitude'], ny_df.index):
    distance, closest_point_index = closest_postcode([lat,lng], coordinates[~coordinates.index.isin([idx])])
    ny_df.at[idx,'Distance'] = np.int64(distance//2) #use the half of the disstance to avoid overlapping
    
ny_df.head(5)

,City,Neighborhood,Latitude,Longitude,Distance
0,NY,Wakefield,40.894705,-73.847201,565.0
1,NY,Co-op City,40.874294,-73.829939,481.0
2,NY,Eastchester,40.887556,-73.827806,742.0
3,NY,Fieldston,40.895437,-73.905643,388.0
4,NY,Riverdale,40.890834,-73.912585,388.0


In [40]:
la_df2.Distance.mean() # use 1000 radius for LA venue 

1023.3089887640449

In [41]:
ny_df.Distance.mean() # use 500 radius for NY venue 

533.1307189542483

In [45]:
CLIENT_ID = '44ZLD3KIUOL5KBS4AI2EUNRKQK1UTZPPAF0KINNMYNRMRMWT' 
CLIENT_SECRET = 'VUD3XCMVEGUVSN1Z2IWIHNTKBTCM5STDKPOC1P002GBJCWFX' 
VERSION = '20200320' 

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius, LIMIT):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
ny_venues = getNearbyVenues(names=ny_df['Neighborhood'],
                                   latitudes=ny_df['Latitude'],
                                   longitudes=ny_df['Longitude'],
                                   radius = 500,
                                   LIMIT = 100
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [51]:
la_venues = getNearbyVenues(names=la_df2['Neighborhood'],
                                   latitudes=la_df2['Latitude'],
                                   longitudes=la_df2['Longitude'],
                                   radius = 1000,
                                   LIMIT = 100
                                  )

Los Angeles(South Los Angeles), Florence-Graham
Los Angeles(Southeast Los Angeles, Watts)
Los Angeles(South Los Angeles, Southeast Los Angeles)
Los Angeles(Hancock Park, Rampart Village, Virgil Village, Wilshire Center, Windsor Square)
Los Angeles(Hancock Park, Koreatown, Wilshire Center, Wilshire Park, Windsor Square)
Los Angeles(Byzantine-Latino Quarter, Harvard Heights, Koreatown, Pico Heights)
Los Angeles(Southeast Los Angeles, Univerity Park)
Los Angeles(Baldwin Hills, Crenshaw, Leimert Park)
Los Angeles(Hancock Park, Wilshire Center, Windsor Square)
Los Angeles(Southeast Los Angeles)
Los Angeles(Downtown Civic Center, Chinatown, Arts District, Bunker Hill, Historic Core, Little Tokyo)
Los Angeles(Downtown Central, Downtown Fashion District)
Los Angeles(Downtown Historic Core, Arts District)
Los Angeles(Dowtown Fashion District, South Park-South)
Los Angeles(West Adams)
Los Angeles(Downtown Bunker Hill, City West, South Park-North)
Los Angeles(Jefferson Park, Leimert Park)
Los Ang

In [96]:
ny_df.head()

,City,Neighborhood,Latitude,Longitude,Distance
0,NY,Wakefield,40.894705,-73.847201,565.0
1,NY,Co-op City,40.874294,-73.829939,481.0
2,NY,Eastchester,40.887556,-73.827806,742.0
3,NY,Fieldston,40.895437,-73.905643,388.0
4,NY,Riverdale,40.890834,-73.912585,388.0


In [162]:
# onehot encoding for venue category then group dataframe by neiborhood by taking mean of each category

ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()

ny_lat_long = ny_df.drop(['Distance'], axis = 1)
ny_grouped =pd.merge(ny_lat_long, ny_grouped, on ='Neighborhood',how='left')
ny_grouped.head()
ny_grouped.head()


,City,Neighborhood,Latitude,Longitude,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstan

In [161]:
# onehot encoding for venue category then group dataframe by neiborhood by taking mean of each category

la_onehot = pd.get_dummies(la_venues[['Venue Category']], prefix="", prefix_sep="")
la_onehot['Neighborhood'] = la_venues['Neighborhood'] 
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]
la_grouped = la_onehot.groupby('Neighborhood').mean().reset_index()

la_lat_long = la_df2.drop(['Distance'], axis = 1)
la_grouped =pd.merge(la_lat_long, la_grouped, on ='Neighborhood',how='left')
la_grouped.head()
la_grouped.head()



,City,Neighborhood,Latitude,Longitude,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Carpet Store,Casino,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus School,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Baseball Diamond,College Bookstore,College Cafeteria,College Classroom,College Quad,College Residence Hall,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Re

## 3. Methodology 
#### a.Explanatory Analysis

In [107]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="New York")
location = geolocator.geocode(address)
ny_latitude = location.latitude
ny_longitude = location.longitude
map_ny = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=10)

# add markers to map
for nyt, lng, neighborhood in zip(ny_grouped['Latitude'], ny_grouped['Longitude'],ny_grouped['Neighborhood']):
    nybel = '{}'.format(neighborhood)
    nybel = folium.Popup(nybel, parse_html=True)
    folium.CircleMarker(
        [nyt, lng],
        radius=5,
        popup=nybel,
        color='Green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

In [109]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="Los Angeles")
location = geolocator.geocode(address)
la_latitude = location.latitude
la_longitude = location.longitude

map_la = folium.Map(location=[la_latitude, la_longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(la_grouped['Latitude'], la_grouped['Longitude'],la_grouped['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la


#### b. Clustering

In [179]:
# concatenate LA / NY data for clustering
nyla_grouped = pd.concat([ny_grouped, la_grouped], ignore_index = True, keys = None,sort=False)
nyla_grouped = nyla_grouped.replace(np.nan,0)
nyla_grouped.head()

,City,Neighborhood,Latitude,Longitude,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstan

In [180]:
# clustering 
kclusters = 15

nyla_grouped_clustering = nyla_grouped.drop(['Neighborhood','City','Latitude','Longitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyla_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
nyla_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
nyla_grouped.head()

,Cluster Labels,City,Neighborhood,Latitude,Longitude,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Rest

In [181]:
#  group dataframe by labels by taking mean of each category
nyla_clusters = nyla_grouped.drop(['City','Neighborhood','Latitude','Longitude'],axis = 1)
nyla_clusters = nyla_clusters.groupby('Cluster Labels').mean().reset_index()
nyla_clusters.head()

,Cluster Labels,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife 

In [182]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [200]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cluster Labels']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nyla_topvenue = pd.DataFrame(columns=columns)
nyla_topvenue['Cluster Labels'] = nyla_clusters['Cluster Labels']

for ind in np.arange(nyla_clusters.shape[0]):
    nyla_topvenue.iloc[ind, 1:] = return_most_common_venues(nyla_clusters.iloc[ind, :], num_top_venues)

nyla_topvenue

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,0,Coffee Shop,Pizza Place,Bar,Italian Restaurant,American Restaurant,Mexican Restaurant,Café
1,1,Park,Water Park,Gay Bar,Food Truck,Fountain,French Restaurant,Fried Chicken Joint
2,2,Scenic Lookout,Rock Climbing Spot,Water Park,Gay Bar,Fountain,French Restaurant,Fried Chicken Joint
3,3,Deli / Bodega,Beach,Italian Restaurant,Bus Stop,Bagel Shop,Dog Run,Chinese Restaurant
4,4,Pizza Place,Deli / Bodega,Chinese Restaurant,Pharmacy,Grocery Store,Bank,Italian Restaurant
5,5,Sculpture Garden,Water Park,Gay Bar,Food Truck,Fountain,French Restaurant,Fried Chicken Joint
6,6,Pool,Baseball Field,Bus Stop,Convenience Store,General College & University,French Restaurant,Fried Chicken Joint
7,7,Beach,Water Park,General College & University,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
8,8,Donut Shop,Sandwich Place,Pharmacy,Grocery Store,Caribbean Restaurant,Bank,Bus Station
9,9,Fast Food Restaurant,Convenience Store,Baseball Field,Sandwich Place,Wings Joint,Sushi Restaurant,Deli / Bodega


## 4. Results

#### a. 101 Neighbor hoods from NY/ 86 from LA are Cluster 0. Based on top 5 venues, it looks like Cluster 0 is food district with lots of Coffee shop, Pizza Place, Bars and Italian Restaurants.

Example: Bayside, Dumbo, Flatiron, Gramercy, etc. (from NY) Beverly Hills, Gardena, Long Beach, Signal Hill, etc. (from LA)

#### b. 72 Neighbor hoods from NY/ 2 from LA are Cluster 4. Based on top 5 venues, it looks like Cluster 4 is also food district with lots of Pizza Place, Deli / Bodega and Chinese Restaurant

Example: Crown Heights ,Norwood, Queensboro Hill, Sunnyside, etc. (from NY) South Los Angeles, City of Industry , etc. (from LA)

#### c. 76 Neighbor hoods from NY/ 7 from LA are Cluster 8. Based on top 5 venues, it looks like Cluster 8 is popular place for commuters since the district is full of quick-prepared food (Donut Shop/Sandwich Place),	Pharmacy and	Grocery Store

Example: Brownsville, Coney Island, Forest Hills, Prospect Park South, etc. (from NY) , Compton East Rancho Dominguez, La Mirada , Hyde Park , etc. (from LA)

#### d. 11 Neighbor hoods from NY/ 64 from LA are Cluster 11. Based on top 5 venues, it looks like Cluster 8 is popular place for commuters since the district is full of quick-prepared food (Mexican Restaurant, Fast Food Restaurant, Pizza Place , Sandwich Place) and Convenience Store. 

Example: East Harlem, Flatbush, Kingsbridge, Stapleton, etc. (from NY)  Carson, Downey, Inglewood, Montebello, etc. (from LA)

In [197]:
ny_only['Cluster Labels'].value_counts(sort=False)

0     101
1       2
3      28
4      72
5       1
6       2
7       1
8      76
9       1
10      1
11     11
12      8
13      1
14      1
Name: Cluster Labels, dtype: int64

In [198]:
la_only['Cluster Labels'].value_counts(sort=False)

0     86
2      1
3      1
4      2
8      7
9     15
11    64
13     2
Name: Cluster Labels, dtype: int64

In [197]:
ny_only['Cluster Labels'].value_counts(sort=False)

0     101
1       2
3      28
4      72
5       1
6       2
7       1
8      76
9       1
10      1
11     11
12      8
13      1
14      1
Name: Cluster Labels, dtype: int64

In [198]:
la_only['Cluster Labels'].value_counts(sort=False)

0     86
2      1
3      1
4      2
8      7
9     15
11    64
13     2
Name: Cluster Labels, dtype: int64

In [189]:
ny_only = nyla_grouped.loc[nyla_grouped.City == 'NY']

ny_map_clusters = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_only['Latitude'], ny_only['Longitude'],ny_only['Neighborhood'], ny_only['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(ny_map_clusters)
       
ny_map_clusters


In [190]:
la_only = nyla_grouped.loc[nyla_grouped.City == 'LA']

la_map_clusters = folium.Map(location=[la_latitude, la_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_only['Latitude'], la_only['Longitude'],la_only['Neighborhood'], la_only['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(la_map_clusters)
       
la_map_clusters

## 5. Discussion / Conclusion

### It is noticeable that both LA/NY have neighborhoods who are categorized into a same Cluster, based on it's venue information. If you are a New Yorker and wants to find a new place in LA (or vice versa) you can refer to this project to find a neighborhood which is similar to your comfort zone. I believe application for this project is bondless. Beyond LA vs. NY, I hope all people who are planning to settle down in a new place to have a soft-landing with the help from this project. 